In [15]:
import io
import torch
import torch.nn as nn
import numpy as np
import chess.pgn
from chess import Board
import torch.nn.functional as F

from utils.datasets import EVAL_HDF_Dataset

dataset = EVAL_HDF_Dataset('/data')
print(dataset[400])
print(dataset[1])

(tensor([[10,  6,  9, 12, 11,  9,  8, 10],
        [ 7,  7,  7,  6,  6,  6,  7,  6],
        [ 6,  6,  8,  6,  6,  6,  6,  7],
        [ 6,  6,  4,  6,  7,  6,  6,  6],
        [ 6,  6,  6,  7,  6,  6,  6,  6],
        [ 6,  6,  6,  5,  6,  4,  6,  6],
        [ 5,  5,  5,  6,  5,  6,  5,  5],
        [ 2,  6,  3,  0,  1,  3,  6,  2]]), tensor(-590))
(tensor([[10,  8,  9, 11, 12,  9,  8, 10],
        [ 7,  7,  7,  7,  7,  7,  7,  7],
        [ 6,  6,  6,  6,  6,  6,  6,  6],
        [ 6,  6,  6,  6,  6,  6,  6,  6],
        [ 6,  6,  6,  6,  6,  6,  6,  6],
        [ 6,  6,  6,  6,  6,  4,  6,  6],
        [ 5,  5,  5,  5,  5,  5,  5,  5],
        [ 2,  4,  3,  1,  0,  3,  6,  2]]), tensor(24))


In [3]:
print(dataset[1])

(tensor([[10,  8,  9, 11, 12,  9,  8, 10],
        [ 7,  7,  7,  7,  7,  7,  7,  7],
        [ 6,  6,  6,  6,  6,  6,  6,  6],
        [ 6,  6,  6,  6,  6,  6,  6,  6],
        [ 6,  6,  6,  6,  6,  6,  6,  6],
        [ 6,  6,  6,  6,  6,  4,  6,  6],
        [ 5,  5,  5,  5,  5,  5,  5,  5],
        [ 2,  4,  3,  1,  0,  3,  6,  2]]), tensor(24))


In [12]:
def get_board(pgn, move):
    game = chess.pgn.read_game(io.StringIO(pgn))
    board = Board()
    # catch board up on game to present
    for past_move in list(game.mainline_moves()):
        board.push(past_move)
    # push the move to score
    board.push_san(move)
    return board

In [13]:
PIECE_CHARS = "♔♕♖♗♘♙⭘♟♞♝♜♛♚"

def encode_board(board: Board) -> np.array:
    # String-encode the board.
    # If board.turn = 1 then it is now white's turn which means this is a potential move
    # being contemplated by black, and therefore we reverse the char order to rotate the board
    # for black's perspective
    # If board.turn = 0 then it is now black's turn which means this is a potential move
    # being contemplated by white, and therefore we leave the char order as white's perspective.
    # Also reverse PIECE_CHARS indexing order if black's turn to reflect "my" and "opponent" pieces.
    step = 1 - 2 * board.turn
    unicode = board.unicode().replace(' ','').replace('\n','')[::step]
    return np.array([PIECE_CHARS[::step].index(c) for c in unicode], dtype=int).reshape(8,8)



In [15]:
pgn = "1.e4 a6 2.Bc4 "
move = "a5"
board = get_board(pgn, move)
board_tensor = torch.tensor(encode_board(board)).unsqueeze(0)
print(board)
print(board_tensor)

r n b q k b n r
. p p p p p p p
. . . . . . . .
p . . . . . . .
. . B . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K . N R
tensor([[[10,  8,  6, 12, 11,  9,  8, 10],
         [ 7,  7,  7,  6,  7,  7,  7,  7],
         [ 6,  6,  6,  6,  6,  6,  6,  6],
         [ 6,  6,  6,  7,  6,  9,  6,  6],
         [ 6,  6,  6,  6,  6,  6,  6,  5],
         [ 6,  6,  6,  6,  6,  6,  6,  6],
         [ 5,  5,  5,  5,  5,  5,  5,  6],
         [ 2,  4,  3,  0,  1,  3,  4,  2]]])


In [16]:
print(board.piece_at(chess.square(0, 0)))


R


In [116]:
FEATURES_DIM = 64 * (64 * 10 + 1)

EXPECTED_OUTPUT_DIM = 2 * FEATURES_DIM
print("features dim:", FEATURES_DIM)
print("expected output dim:", EXPECTED_OUTPUT_DIM)

PIECE_TYPES = [chess.QUEEN, chess.ROOK, chess.BISHOP, chess.KNIGHT, chess.PAWN]
COLORS = [chess.BLACK, chess.WHITE]

def encode_board(board: Board) -> np.array:
    opps = np.zeros(FEATURES_DIM)
    mys = np.zeros(FEATURES_DIM)

    if board.turn:
        my_color = chess.WHITE
        opp_color = chess.BLACK 
    else:
        my_color = chess.BLACK  
        opp_color = chess.WHITE

    my_king, opp_king = board.king(my_color), board.king(opp_color)
    print("my_king", my_king, "opp_king", opp_king)

    for i in range(8):
        for j in range(8):
            piece = board.piece_at(chess.square(i, j))
            if piece and piece.piece_type != chess.KING:
                piece_index = PIECE_TYPES.index(piece.piece_type)
                print("piece index:", piece_index, "piece:", piece)
                if piece.color == opp_color:
                    opps[(opp_king * (64 * 10 + 1)) + (piece_index * 64) + (i * 8) + j] = 1
                else:
                    mys[(my_king * (64 * 10 + 1)) + (piece_index * 64) + (i * 8) + j] = 1

    # for piece_type in enumerate(PIECE_TYPES):
    #     for color in enumerate(COLORS):
    #         piece = chess.Piece(piece_type[1], color[1])

    #         piece_index = piece.piece_type * 8 + piece.color
    #         blacks[black_king * 10 + piece_index] = 1
    #         whites[white_king * 10 + piece_index] = 1

    # for i in range(8):
    #     for j in range(8):
    #         piece = board.piece_at(chess.square(i, j))
    #         # print(piece)
    #         if piece == chess.Piece(chess.KING, chess.BLACK):

    #             king = i * 8 + j
    #             for color in COLORS:
    #                 for piece_type in PIECE_TYPES:
    #                     piece = chess.Piece(piece_type, color)
    #                     piece_index = piece.piece_type * 8 + piece.color
    #                     blacks[king * 10 + piece_index] = 1
    #                     whites[king * 10 + piece_index] = 1

    return np.concatenate((opps, mys))


features dim: 41024
expected output dim: 82048


In [117]:
encoded = encode_board(board)   
print(encoded)
print(encoded.shape)

print(encoded.max())

count = 0
for i in encoded:
    if i == 1:
        count += 1
print(count)

my_king 4 opp_king 60
piece index: 1 piece: R
piece index: 4 piece: P
piece index: 4 piece: p
piece index: 1 piece: r
piece index: 3 piece: N
piece index: 4 piece: P
piece index: 4 piece: p
piece index: 3 piece: n
piece index: 2 piece: B
piece index: 4 piece: P
piece index: 2 piece: B
piece index: 4 piece: p
piece index: 2 piece: b
piece index: 0 piece: Q
piece index: 4 piece: P
piece index: 4 piece: p
piece index: 0 piece: q
piece index: 4 piece: P
piece index: 4 piece: p
piece index: 4 piece: P
piece index: 4 piece: p
piece index: 2 piece: b
piece index: 3 piece: N
piece index: 4 piece: P
piece index: 4 piece: p
piece index: 3 piece: n
piece index: 1 piece: R
piece index: 4 piece: P
piece index: 4 piece: p
piece index: 1 piece: r
[0. 0. 0. ... 0. 0. 0.]
(82048,)
1.0
30


In [26]:
FEATURES_DIM = 64 * (64 * 10 + 1)

EXPECTED_OUTPUT_DIM = 2 * FEATURES_DIM
print("features dim:", FEATURES_DIM)
print("expected output dim:", EXPECTED_OUTPUT_DIM)

OPP_PIECE_TYPES = [11, 10, 9, 8, 7]
MY_PIECE_TYPES = [1, 2, 3, 4, 5]

def encode_dataset(d):
    my_king, opp_king = None, None

    for i in range(8):
        for j in range(8):
            piece = d[i][j]
            if piece == 0:
                my_king = (i, j)
            elif piece == 12:
                opp_king = (i, j)

    opps = np.zeros(FEATURES_DIM)
    mys = np.zeros(FEATURES_DIM)

    for i in range(8):
        for j in range(8):
            piece = d[i][j]

            if piece != 6 and piece != 12 and piece != 0:
                if piece in OPP_PIECE_TYPES:
                    piece_index = OPP_PIECE_TYPES.index(piece)
                    opps[(opp_king[0]*8+opp_king[1]) * (64 * 10 + 1) + (piece_index * 64) + (i * 8) + j] = 1
                elif piece in MY_PIECE_TYPES:
                    piece_index = MY_PIECE_TYPES.index(piece)
                    mys[(my_king[0]*8+my_king[1]) * (64 * 10 + 1) + (piece_index * 64) + (i * 8) + j] = 1
                    
    return np.concatenate((opps, mys))


features dim: 41024
expected output dim: 82048


In [28]:
print(dataset[1])
encoded_board = encode_dataset(dataset[1][0])
 
print(encoded_board)
print(encoded_board.shape)

print(encoded_board.max())

count = 0
for i in encoded_board:
    if i == 1:
        count += 1
print(count)

(tensor([[10,  8,  9, 11, 12,  9,  8, 10],
        [ 7,  7,  7,  7,  7,  7,  7,  7],
        [ 6,  6,  6,  6,  6,  6,  6,  6],
        [ 6,  6,  6,  6,  6,  6,  6,  6],
        [ 6,  6,  6,  6,  6,  6,  6,  6],
        [ 6,  6,  6,  6,  6,  4,  6,  6],
        [ 5,  5,  5,  5,  5,  5,  5,  5],
        [ 2,  4,  3,  1,  0,  3,  6,  2]]), tensor(24))
[0. 0. 0. ... 0. 0. 0.]
(82048,)
1.0
30
